## Probelm 1 ##

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob

# I tired using an iphone but due to high res (12MP) it was taking too long
# Thus I decided to use my webcam and forced it to a 0.9MP (1280x720) and it speeded up the process  

CHECKERBOARD_DIM = (7,10)


# Make corresponding 3D world points (assuming each square is 1 unit away)
# (0,0,0) (1,0,0) ... (6,0,0) (0,1,0)...(6,1,0) (0,9,0)..(6,9,0) Only inner points of checker
objectPoints_3D = np.zeros((1, CHECKERBOARD_DIM[0]* CHECKERBOARD_DIM[1],3), np.float32)
objectPoints_3D[0, :, :2] = np.mgrid[0:CHECKERBOARD_DIM[0],0:CHECKERBOARD_DIM[1]].T.reshape(-1, 2)

#temp = list(objectPoints_3D[0,:,0])
#objectPoints_3D[0,:,0] = objectPoints_3D[0,:,1]
#objectPoints_3D[0,:,1] = temp
#objectPoints_3D = objectPoints_3D[:,::-1]

# array to store point pairs 
points_3D = []
points_2D = []

image = glob.glob('checker*.jpg')
# iterate trough all 21 checker images
for filename in image:
    img = cv.imread(filename)
 
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD_DIM,None)
    
    points_3D.append(objectPoints_3D)
    points_2D.append(corners)

    #cv.drawChessboardCorners(img, CHECKERBOARD_DIM,corners, ret)
    #cv.imshow('img', img)
    #cv.waitKey(0)
    #cv.destroyAllWindows()

# Once we have all point pairs I use the built in function of OpenCV to find intrinsic and extrinsic matrix for each Image
ret, matrix, distortion, r_vecs, t_vecs = cv.calibrateCamera(points_3D, points_2D, gray.shape[::-1], None, None)

# Displaying Intrinsic Matrix (focal length and scale)
print("Camera Intrinsic matrix:")
print(matrix)

r_mat = []
# Displaying unique extrinsic matrix for each images (rotation matix and translation vector)
# OpenCV returns a rotation vector and must be converted to rotation matrix
for i in range(len(r_vecs)):
    r_mat_i = np.zeros((3,3))
    cv.Rodrigues(r_vecs[i],r_mat_i)
    r_mat.append(r_mat_i)
    print("\nExtrensic Matrix for Image {0} :-".format(i+1))
    print("\n Rotation Matrix:")
    print(r_mat[i])
    print("\n Translation Vectors:")
    #t_vecs[i] = tuple(list(t_vecs[i]).insert(3,1))
    print(t_vecs[i])
    print("-------------------------------------------------------------------")
    
# OPenCV or python has no built-in function to show extrinsinc param in one plot. 
# Thus I used Matlab to make the graph using Camera Calibration tool
# The results for focal length and intrinsic matrix match with certain margin of error.
# however Matlab has origin of corners in top left and OpenCv in top right
# Causing some extrinsinc param to differ. I will just dispaly the plot according to matlab.

pattern = cv.imread("patternCentric.png")
plt.imshow(pattern)
plt.show()

camera = cv.imread("cameraCentric.png")
plt.imshow(camera)
plt.show()

## Problem 2 ##

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines

def midpoint (leftRay, rightRay , l_R_r , l_t_r):
    # a*leftRay + c (leftRay X l_R_r*rightRay) = b*l_R_r*rightRay + l_t_r
    # a*leftRay + c (q) = b*l_R_r*rightRay + l_t_r
    
    q = np.cross(leftRay,np.matmul(l_R_r,rightRay)) # segmet between right and left ray
    q = q/np.linalg.norm(q) # normalize the cross product or segemtn between right ray and left ray
    
    # gattering everything to one side
    # a*leftRay + c (q) - b*l_R_r*rightRay = l_t_r
    # A [a b c].T = l_t_r
    A = np.vstack((leftRay,-1*np.matmul(l_R_r,rightRay),q)).T
    
    # [a b c].T = inv(A) * l_t_r
    sol = np.matmul(np.linalg.inv(A), l_t_r) 
    
    a = sol[0]
    b = sol[1]
    c = sol[2]
    
    # mid point is a*leftRay + 0.5*c(q) --> Halfway between right camera ray and left camera ray
    midpoint_sol = a*leftRay + c*0.5*q
    
    return midpoint_sol
    
    

def reconstruct3D (leftRays, rightRays, l_R_r , l_t_r , w_M_l):
    
    points_3D = np.zeros((len(leftRays[0]),len(w_M_l)))
    
    for i in range(len(leftRays[0])):
        # Find mid point of two rays wrt to left camera
        midpoint_wrt_left = midpoint(leftRays[:,i].T, rightRays[:,i], l_R_r, l_t_r)
        midpoint_wrt_left = np.hstack((midpoint_wrt_left,[1]))
        
        # Find the mid point wrt to world
        midpoint_wrt_world = np.matmul(w_M_l,midpoint_wrt_left) 

        # Add it to output 
        points_3D[i] = midpoint_wrt_world.T
        
    return points_3D

def plot3D (pts,title):
    x = pts[:,0]
    y = pts[:,1]
    z = pts[:,2]
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    
    ax.plot([x[0],x[1]],[y[0],y[1]],[z[0],z[1]],'b')
    ax.plot([x[1],x[2]],[y[1],y[2]],[z[1],z[2]],'b')
    ax.plot([x[2],x[3]],[y[2],y[3]],[z[2],z[3]],'b')
    ax.plot([x[3],x[0]],[y[3],y[0]],[z[3],z[0]],'b')
    
    ax.plot([x[4],x[5]],[y[4],y[5]],[z[4],z[5]],'b')
    ax.plot([x[5],x[6]],[y[5],y[6]],[z[5],z[6]],'b')
    ax.plot([x[6],x[7]],[y[6],y[7]],[z[6],z[7]],'b')
    ax.plot([x[7],x[4]],[y[7],y[4]],[z[7],z[4]],'b')
    
    ax.plot([x[0],x[4]],[y[0],y[4]],[z[0],z[4]],'b')
    ax.plot([x[1],x[5]],[y[1],y[5]],[z[1],z[5]],'b')
    ax.plot([x[2],x[6]],[y[2],y[6]],[z[2],z[6]],'b')
    ax.plot([x[3],x[7]],[y[3],y[7]],[z[3],z[7]],'b')
    plt.title(title)
    plt.show()

def plot2D (pts,title):
    x = pts[:,0]
    y = pts[:,1]
    z = pts[:,2]
    
    fig = plt.figure()
    ax = plt.axes()
    
    ax.plot([x[0],x[1]],[y[0],y[1]],'b')
    ax.plot([x[1],x[2]],[y[1],y[2]],'b')
    ax.plot([x[2],x[3]],[y[2],y[3]],'b')
    ax.plot([x[3],x[0]],[y[3],y[0]],'b')
    
    ax.plot([x[4],x[5]],[y[4],y[5]],'b')
    ax.plot([x[5],x[6]],[y[5],y[6]],'b')
    ax.plot([x[6],x[7]],[y[6],y[7]],'b')
    ax.plot([x[7],x[4]],[y[7],y[4]],'b')
    
    ax.plot([x[0],x[4]],[y[0],y[4]],'b')
    ax.plot([x[1],x[5]],[y[1],y[5]],'b')
    ax.plot([x[2],x[6]],[y[2],y[6]],'b')
    ax.plot([x[3],x[7]],[y[3],y[7]],'b')
    plt.title(title)
    plt.show()

In [ ]:
K = np.array([ [-100, 0    , 200],
               [0   , -100 , 200],
               [0   , 0    , 1]])

M_EXT_L = np.array([[0.707  , 0.707, 0, -3  ],
                  [-0.707 , 0.707, 0, -0.5],
                  [0,        0,    1,  3  ]])

M_EXT_R = np.array([[0.866  , -0.5,  0, -3  ],
                  [0.5   , 0.866, 0, -0.5],
                  [0      , 0    , 1,  3  ]])

PTS = np.array([[2,0,0],
                [3,0,0],
                [3,1,0],
                [2,1,0],
                [2,0,1],
                [3,0,1],
                [3,1,1],
                [2,1,1],
                [2.5,0.5,2]])

 
plot3D(PTS,'Original Cube')  

LEFT_PIXELS = np.zeros_like(PTS)
RIGHT_PIXELS = np.zeros_like(PTS)

# calculte camera cordinates using right camera and left camera refrence frame
for i in range(len(PTS)) :
    pixel = np.matmul(K,np.matmul(M_EXT_L,np.hstack((PTS[i],[1]))))
    LEFT_PIXELS[i] = pixel / pixel[2]
    pixel = np.matmul(K,np.matmul(M_EXT_R,np.hstack((PTS[i],[1]))))
    RIGHT_PIXELS[i] = pixel / pixel[2]

plot2D(LEFT_PIXELS,'Left Camera')
plot2D(RIGHT_PIXELS,'Right Camera')

# Assuming only left and right camera pixels are known
# Reconstruct 3D points

# Find ray for each point for each camera

LEFT_RAY = np.matmul(np.linalg.inv(K),LEFT_PIXELS.T)
RIGHT_RAY = np.matmul(np.linalg.inv(K),RIGHT_PIXELS.T)

L_M_W = np.vstack((M_EXT_L,[0,0,0,1]))
W_M_L = np.linalg.inv(L_M_W)

R_M_W = np.vstack((M_EXT_R,[0,0,0,1]))
W_M_R = np.linalg.inv(R_M_W)

# Matrix that turns all pixels in right camera to left camera
L_M_R = np.matmul(L_M_W , W_M_R)
L_ROTATION_R = L_M_R [0:3,0:3]
L_TRANSLATION_R = L_M_R [0:3,3:4]

reconst_3D = reconstruct3D(LEFT_RAY,RIGHT_RAY,L_ROTATION_R,L_TRANSLATION_R,W_M_L)
reconst_3D = reconst_3D[:,0:3] 

plot3D(reconst_3D,"Reconstructed Cube")  

print("\nGround Truth (Orginal Points) :- ")
print(PTS)

print("\nReconstructed Points :-")
print(reconst_3D)

print("\nAbsolute error btw points :-")
print(abs(PTS - reconst_3D))


# proj. error = || Origninal - Reconstructed ||

print("\nProjection Error :-")
print(np.linalg.norm(PTS - reconst_3D))